In [1]:
import pandas as pd
import numpy as np
from HFC32_bmimAc_PR import configuration
from pyomo.environ import (ConcreteModel,
                           SolverFactory,
                           units as pyunits)

import idaes.logger as idaeslog
import pyomo.contrib.parmest.parmest as parmest

import pytest

from idaes.core import FlowsheetBlock
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

from idaes.core.util.model_statistics import degrees_of_freedom

In [2]:
def PR_model(data):
    """
    Function which returns initilized model.

    Args:
        data: pandas DataFrame with data

    Returns:
        initialized model
    """
    eps = 0.1
    init_kappa_2_1A = -0.54
    init_kappa_1_2A = -0.03
    m = ConcreteModel()

    m.fs = FlowsheetBlock(default={"dynamic": False})

    m.fs.properties = GenericParameterBlock(default=configuration)

    m.fs.state_block = m.fs.properties.state_block_class(
        default={"parameters": m.fs.properties,
                 "defined_state": True})
    x = float(data['x_R32'])+eps
    m.fs.state_block.flow_mol.fix(1)
    m.fs.state_block.temperature.fix(float(data["temperature"]))
    m.fs.state_block.pressure.fix(float(data["pressure"]))
    m.fs.state_block.mole_frac_comp['bmimAc'].fix(1-x)
    m.fs.state_block.mole_frac_comp['R32'].fix(x)

    # parameter - kappa_A_ij (set at 0.3, 0 if i=j)
    m.fs.properties.PR_kappa_A['bmimAc','bmimAc'].fix(0)
    m.fs.properties.PR_kappa_A['bmimAc', 'R32'].fix(init_kappa_2_1A)
    m.fs.properties.PR_kappa_A['R32', 'R32'].fix(0)
    m.fs.properties.PR_kappa_A['R32', 'bmimAc'].fix(init_kappa_1_2A)
    m.fs.properties.PR_kappa_B['bmimAc', 'bmimAc'].fix(0)
    m.fs.properties.PR_kappa_B['bmimAc', 'R32'].fix(0)
    m.fs.properties.PR_kappa_B['R32', 'R32'].fix(0)
    m.fs.properties.PR_kappa_B['R32', 'bmimAc'].fix(0)
    m.fs.properties.PR_kappa_C['bmimAc', 'bmimAc'].fix(0)
    m.fs.properties.PR_kappa_C['bmimAc', 'R32'].fix(0)
    m.fs.properties.PR_kappa_C['R32', 'R32'].fix(0)
    m.fs.properties.PR_kappa_C['R32', 'bmimAc'].fix(0)
    # Initialize the flash unit
    m.fs.state_block.initialize(outlvl=idaeslog.CRITICAL)

    # Fix the state variables on the state block
    m.fs.state_block.pressure.unfix()
    m.fs.state_block.mole_frac_comp['bmimAc'].unfix()
    m.fs.state_block.mole_frac_comp['R32'].unfix()
    m.fs.state_block.temperature.fix(float(data["temperature"]))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'R32'].fix(float(data['x_R32']))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'bmimAc'].fix(float(data['x_bmimAc']))
    m.fs.state_block.mole_frac_comp['R32'].fix(float(data['x_R32'])+eps)
    m.fs.state_block.mole_frac_comp['bmimAc'].unfix()

    # Set bounds on variables to be estimated
    m.fs.properties.PR_kappa_A['bmimAc', 'R32'].setlb(-5)
    m.fs.properties.PR_kappa_A['bmimAc', 'R32'].setub(5)

    m.fs.properties.PR_kappa_A['R32', 'bmimAc'].setlb(-5)
    m.fs.properties.PR_kappa_A['R32', 'bmimAc'].setub(5)

    # Return initialized flash model
    return m

def SSE(m, data):
    """
    returns objective function expresion.

    Args:
        m: model
        data: pandas DataFrame with data

    Returns:
        objectuve function scaled expresion
    """
    expr = ((float(data["pressure"]) - m.fs.state_block.pressure)**2)

    return expr * 1e-6

variable_name = ["fs.properties.PR_kappa_A['bmimAc', 'R32']",
                 "fs.properties.PR_kappa_A['R32', 'bmimAc']"]


In [3]:
data = pd.read_csv('R32_bmimAc.csv')

print(data)

   temperature  pressure  x_R32  x_bmimAc
0       298.15     50000  0.054     0.946
1       298.15    100000  0.097     0.903
2       298.15    200000  0.161     0.839
3       298.15    400000  0.268     0.732
4       298.15    600000  0.357     0.643
5       298.15    800000  0.438     0.562
6       298.15   1000000  0.510     0.490


In [4]:
N = 100 #number of samples
results_params = np.zeros((N,2))
results_obj = np.zeros((N,1))

st_dev_T = 0.01 #C
st_dev_P = 9771 #MPa
st_dev_x = 0.10 #mole fraction

for i in range(N):
    print(i)
    noise_x = np.random.normal(loc=0,scale=st_dev_x)
    print(noise_x)
    data_ = data.copy()
    data_["x_bmimAc"] += data_["x_bmimAc"] * noise_x
    data_.loc[data_['x_bmimAc'] <= 0.0001, 'x_bmimAc'] = 0.0001
    data_.loc[data_['x_bmimAc'] >= 0.9999 , 'x_bmimAc'] = 0.9999      

    try:
        pest = parmest.Estimator(PR_model, data_, variable_name, SSE, tee=False)

        obj_value, parameters = pest.theta_est()

        for k,v in parameters.items():
            print(k, "=", v)

        line = 0

        for k,v in parameters.items():
            results_params[i,line] = v
            line += 1
        results_obj[i] = obj_value

    except ValueError:
        results_params[i,0] = 'NaN'
        results_params[i,1] = 'NaN'


0
0.005463207089833234
2022-03-06 21:10:16 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:10:17 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:10:18 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:10:19 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:10:20 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:10:21 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:10:22 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
    model.name="_Q_opt";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a lo

fs.properties.PR_kappa_A[bmimAc,R32] = -0.7366546278282111
fs.properties.PR_kappa_A[R32,bmimAc] = -0.04098771717915124
7
-0.028586677744944264
2022-03-06 21:11:24 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:11:25 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:11:27 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:11:28 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:11:29 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:11:30 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:11:31 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
fs.propert

2022-03-06 21:12:36 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:12:37 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:12:38 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:12:39 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
    model.name="_Q_opt";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
fs.properties.PR_kappa_A[bmimAc,R32] = -0.7719961359786961
fs.properties.PR_kappa_A[R32,bmimAc] = -0.041366646585976566
15
0.022780470822306373
2022-03-06 21:12:43 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:12:44 [INFO] idaes.init.fs.state_block: Property

2022-03-06 21:13:45 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:13:46 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:13:47 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:13:48 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
fs.properties.PR_kappa_A[bmimAc,R32] = -0.5634804270480529
fs.properties.PR_kappa_A[R32,bmimAc] = -0.03197734386330659
22
0.23682874672081003
2022-03-06 21:13:51 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:13:52 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:13:53 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 

2022-03-06 21:15:02 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:15:03 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:15:04 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:15:05 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:15:07 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
fs.properties.PR_kappa_A[bmimAc,R32] = -0.5656834668276425
fs.properties.PR_kappa_A[R32,bmimAc] = -0.038370605957272645
30
0.015576546461477945
2022-03-06 21:15:10 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:15:11 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-0

...

2022-03-06 21:21:02 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:21:03 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:21:04 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:21:05 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:21:06 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:21:07 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:21:08 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
fs.properties.PR_kappa_A[bmimAc,R32] = -0.5652263443232232
fs.properties.PR_kappa_A[R32,bmimAc] = -0.036876725175920326
67
-0.04678385830949082
2022-03-0

fs.properties.PR_kappa_A[bmimAc,R32] = -0.5664538358872753
fs.properties.PR_kappa_A[R32,bmimAc] = -0.0412031577267093
74
-0.11093649911932389
2022-03-06 21:22:19 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:22:20 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:22:21 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:22:22 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:22:23 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:22:24 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:22:26 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
fs.properti

2022-03-06 21:23:30 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:23:31 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:23:33 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:23:34 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
    model.name="_Q_opt";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
fs.properties.PR_kappa_A[bmimAc,R32] = -0.7681344620189271
fs.properties.PR_kappa_A[R32,bmimAc] = -0.041351738156702796
82
0.10416377243094828
2022-03-06 21:23:37 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:23:38 [INFO] idaes.init.fs.state_block: Property 

2022-03-06 21:24:37 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:24:38 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:24:39 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:24:40 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:24:41 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:24:42 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
fs.properties.PR_kappa_A[bmimAc,R32] = -0.5655137171717805
fs.properties.PR_kappa_A[R32,bmimAc] = -0.037802311360896006
89
-0.10075754552395025
2022-03-06 21:24:46 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-0

fs.properties.PR_kappa_A[bmimAc,R32] = -0.8105044184980462
fs.properties.PR_kappa_A[R32,bmimAc] = -0.04143456515334084
96
-0.03890902232143897
2022-03-06 21:25:55 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:25:56 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:25:57 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:25:58 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:25:59 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:26:00 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:26:01 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
fs.propert

In [5]:
print(results_params)

[[-0.70590894 -0.04004464]
 [-0.72807492 -0.04076951]
 [-0.74104356 -0.04108521]
 [-0.74077226 -0.04107993]
 [-0.7863816  -0.04139064]
 [-0.56833668 -0.05290799]
 [-0.73665463 -0.04098772]
 [-0.56400515 -0.03334343]
 [-0.56768538 -0.04727001]
 [-0.7418513  -0.04109842]
 [-0.83965726 -0.04149395]
 [-0.56654446 -0.04157019]
 [-0.56691677 -0.04317777]
 [-0.72681095 -0.04073303]
 [-0.77199614 -0.04136665]
 [-0.71833276 -0.0404407 ]
 [-0.56459256 -0.03497231]
 [-0.72175503 -0.04056038]
 [-0.80811102 -0.04143222]
 [-0.82278111 -0.0414265 ]
 [-0.56743154 -0.04576088]
 [-0.56348043 -0.03197734]
 [-0.83512082 -0.04148089]
 [-0.56407254 -0.03352459]
 [-0.56747605 -0.04601137]
 [-0.71215952 -0.04022175]
 [-0.56382465 -0.03286486]
 [-0.56427951 -0.03408988]
 [-0.5681268  -0.05060999]
 [-0.56568347 -0.03837061]
 [-0.71252016 -0.04024083]
 [-0.56385513 -0.03294502]
 [-0.72421879 -0.04065007]
 [-0.56547634 -0.03767941]
 [-0.70633612 -0.04006045]
 [-0.76377205 -0.04132258]
 [-0.73879497 -0.04105053]
 

In [6]:
results = pd.DataFrame(results_params)
results.to_csv('R32_bmimAc_10_2.csv')